In [1]:
from stable_baselines3 import DQN
from stable_baselines3.dqn import MlpPolicy
from stable_baselines3 import PPO
from typing import TYPE_CHECKING, Any, Callable, Dict, List, Optional, Union
from stable_baselines3.common.callbacks import EvalCallback, BaseCallback
from stable_baselines3.common.vec_env import DummyVecEnv, VecEnv, sync_envs_normalization
from stable_baselines3.common.monitor import Monitor
import gymnasium as gym

In [2]:
from gymnasium_env.envs.f1_env import RewardFunction
from gymnasium_env.envs.f1_env import RewardFunctionPositionChangeByLapsToGo

from racesim.config import *

In [3]:
class F1EnvEvalCallback(EvalCallback):
    def __init__(
        self,
        eval_env: Union[gym.Env, VecEnv],
        callback_on_new_best: Optional[BaseCallback] = None,
        callback_after_eval: Optional[BaseCallback] = None,
        n_eval_episodes: int = 5,
        eval_freq: int = 10000,
        log_path: Optional[str] = None,
        best_model_save_path: Optional[str] = None,
        deterministic: bool = True,
        render: bool = False,
        verbose: int = 1,
        warn: bool = True,
        seed_eval_env: int = 0,
        reward_function: RewardFunction = None,
        grid_config: GridConfig = None
    ):
        super().__init__(eval_env,callback_on_new_best,callback_after_eval,n_eval_episodes,eval_freq,
                         log_path,best_model_save_path,deterministic,render,verbose,warn)
        self.seed_eval_env = seed_eval_env
        self.reward_function = reward_function
        self.grid_config = grid_config

    def _on_step(self) -> bool:
        continue_training = super()._on_step()
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            self.set_env_for_next_eval()
        return continue_training
        

    def set_env_for_next_eval(self) -> None:
        self.eval_env = gym.make("F1Env/Basic-v0", seed=self.seed_eval_env,
                                 reward_function=self.reward_function, grid_config=self.grid_config)

In [4]:
seed = 0
n_seeds = 3
seed_eval = 50
reward_function = RewardFunctionPositionChangeByLapsToGo()
grid_config = GridConfigSortedById()

In [5]:
for i in range(n_seeds):
    print("-----------------\n\n" + " RUN " + str(i) + "\n\n--------------------------")
    env = gym.make("F1Env/Basic-v0", seed=i, reward_function=reward_function, grid_config=grid_config)
    env_to_eval = gym.make("F1Env/Basic-v0", seed=seed_eval, reward_function=reward_function, grid_config=grid_config)

    eval_callback = F1EnvEvalCallback(eval_env=env_to_eval, best_model_save_path="./logs_" + str(i) + "/",
                             log_path="./logs_" + str(i) + "/", n_eval_episodes=20, eval_freq=25000,
                             deterministic=True, render=False, seed_eval_env=seed_eval,
                                      reward_function=reward_function, grid_config=grid_config)

    model_reward_position_change = DQN("MlpPolicy", env, gamma=1, buffer_size=100000, learning_starts=5000,
                                            train_freq=(16,"step"), exploration_initial_eps=0.25, exploration_fraction=0.2,
                                            exploration_final_eps=0.020, learning_rate=0.00005,
                                            batch_size=256,
                                            verbose=0, tensorboard_log="./dqn_r2/", seed=i)
    model_reward_position_change.learn(total_timesteps=1e6, callback=eval_callback)
    model_reward_position_change.save(path="./model_reward_position_change_"+str(i))

-----------------

 RUN 0

--------------------------


/home/usuario/miniconda3/envs/rfl/lib/python3.11/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=25000, episode_reward=-3.99 +/- 1.57
Episode length: 77.00 +/- 0.00
New best mean reward!
Eval num_timesteps=50000, episode_reward=-0.73 +/- 0.80
Episode length: 77.00 +/- 0.00
New best mean reward!
Eval num_timesteps=75000, episode_reward=-0.79 +/- 1.01
Episode length: 77.00 +/- 0.00
Eval num_timesteps=100000, episode_reward=-1.22 +/- 1.31
Episode length: 77.00 +/- 0.00
Eval num_timesteps=125000, episode_reward=-3.99 +/- 1.57
Episode length: 77.00 +/- 0.00
Eval num_timesteps=150000, episode_reward=-3.30 +/- 1.65
Episode length: 77.00 +/- 0.00
Eval num_timesteps=175000, episode_reward=-3.34 +/- 2.14
Episode length: 77.00 +/- 0.00
Eval num_timesteps=200000, episode_reward=-3.10 +/- 1.32
Episode length: 77.00 +/- 0.00
Eval num_timesteps=225000, episode_reward=-3.94 +/- 1.96
Episode length: 77.00 +/- 0.00
Eval num_timesteps=250000, episode_reward=-3.56 +/- 1.97
Episode length: 77.00 +/- 0.00
Eval num_timesteps=275000, episode_reward=-1.38 +/- 3.58
Episode length: 77.00 

In [6]:
# import numpy as np
# from tqdm import tqdm
# import torch
# from stable_baselines3.common.utils import obs_as_tensor
# env_to_test = gym.make("F1Env/Basic-v0")

In [7]:
# agent = model_reward_position_change
# stops = 0

# for episode in tqdm(range(1)):
#     t = 0
#     total_rw = 0
#     obs, info = env_to_test.reset()
#     print(env_to_test.our_driver.tyre.__dict__)
#     # print(obs)
#     # print("------")
#     print("START POSITION: ", env_to_test.our_driver.position)
#     done = False

#     # play one episode
#     while not done:
#         # display(env_to_test.race_simulation.to_df())
#         action, _ = agent.predict(obs, deterministic=True)
#         with torch.no_grad():
#             observation = obs.reshape((-1,) + agent.observation_space.shape)
#             observation = obs_as_tensor(observation, 'cpu')
#             q_values = agent.q_net(observation)
#             # print(q_values)
#         if action != 0:
#             stops+=1
#             print(action, " - laps to go: ", env_to_test.our_driver.laps_to_go, " - position: ", env_to_test.our_driver.position)
#             print(q_values)
#         next_obs, reward, terminated, truncated, info = env_to_test.step(action)
#         # print('REWARDA: ', reward)

#         total_rw += reward

#         done = terminated or truncated
#         obs = next_obs
#         # print(obs)
#         # print("------")
#         t+=1

#     # print("Reward: " , total_rw)
# # display(env_to_test.race_simulation.to_df())
# print("Reward: " , total_rw, " - ", "Stops: ", stops)
# env_to_test.race_simulation.driver_being_controlled.position

  0%|                                                     | 0/1 [00:00<?, ?it/s]

{'compound': 0, 'base_lap_time': 70.0193, 'squared_deg': 0.00281, 'linear_deg': -0.01999, 'used_laps': 0}
START POSITION:  14
2  - laps to go:  71  - position:  13
tensor([[ 0.0537, -0.4632,  0.0647, -0.4645]])
1  - laps to go:  51  - position:  20
tensor([[2.1372, 2.1472, 2.0500, 1.5315]])
2  - laps to go:  40  - position:  14
tensor([[-0.4095, -0.6065, -0.3871, -1.5063]])


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.06it/s]

Reward:  4  -  Stops:  3


10

In [8]:
# env_to_test.race_simulation.to_df()

,id,interval,gap,lapsToGo,lastLapTime,position,potentialLapTime,compound,usedLaps,secondDry
0,6,0.000000,0.000000,0,73.217360,1,73.31914,0,38,True
1,13,0.402896,0.402896,0,73.423376,2,73.53250,0,39,True
2,14,35.195148,35.598044,0,73.742110,3,73.78440,2,45,True
3,7,0.255083,35.853127,0,74.018995,4,74.06536,2,48,True
4,1,0.877605,36.730733,0,74.038773,5,74.08638,2,48,True
5,4,0.203064,36.933797,0,73.946171,6,74.06914,0,42,True
6,16,0.267274,37.201070,0,73.976423,7,72.20032,0,32,True
7,10,0.046089,37.247159,0,73.661027,8,73.70020,2,45,True
8,15,5.169603,42.416762,0,73.521490,9,73.63220,0,40,True
9,19,6.542375,48.959137,0,72.836228,10,72.88560,1,39,True
